<a href="https://colab.research.google.com/github/alok13/Machine-Learning/blob/master/LSTM_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CHARACTER WISE TEXT GENERATION **

In [1]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

TensorFlow 2.x selected.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text=open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
vocab = sorted(set(text))

In [6]:
print ('{} unique characters'.format(len(vocab)))

65 unique characters


**PROCESS THE TEXT**
We need to map strings to a numerical representation. Creating two lookup tables : one mapping characters to numbers and another for numbers to character.


In [0]:
char2idx = {u: i for i,u in enumerate(vocab)}
idx2char =np.array(vocab)

text_as_int =np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [21]:
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [22]:
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [29]:
for aa in idx2char:
   print('  {:4s}: {:3d},'.format(repr(aa), list(idx2char).index(aa)))
   
  

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  'H' :  20,
  'I' :  21,
  'J' :  22,
  'K' :  23,
  'L' :  24,
  'M' :  25,
  'N' :  26,
  'O' :  27,
  'P' :  28,
  'Q' :  29,
  'R' :  30,
  'S' :  31,
  'T' :  32,
  'U' :  33,
  'V' :  34,
  'W' :  35,
  'X' :  36,
  'Y' :  37,
  'Z' :  38,
  'a' :  39,
  'b' :  40,
  'c' :  41,
  'd' :  42,
  'e' :  43,
  'f' :  44,
  'g' :  45,
  'h' :  46,
  'i' :  47,
  'j' :  48,
  'k' :  49,
  'l' :  50,
  'm' :  51,
  'n' :  52,
  'o' :  53,
  'p' :  54,
  'q' :  55,
  'r' :  56,
  's' :  57,
  't' :  58,
  'u' :  59,
  'v' :  60,
  'w' :  61,
  'x' :  62,
  'y' :  63,
  'z' :  64,


In [31]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [34]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

In [36]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(30):
  print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i
z
e
n
:


B
e
f
o
r
e
 
w
e
 
p
r
o
c
e


In [37]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [40]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [42]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:10], target_example[:10])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')
Step    5
  input: 1 (' ')
  expected output: 15 ('C')
Step    6
  input: 15 ('C')
  expected output: 47 ('i')
Step    7
  input: 47 ('i')
  expected output: 58 ('t')
Step    8
  input: 58 ('t')
  expected output: 47 ('i')
Step    9
  input: 47 ('i')
  expected output: 64 ('z')


In [43]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model =build_model(vocab_size=len(vocab),
                   embedding_dim=embedding_dim,
                   rnn_units=rnn_units,
                   batch_size=BATCH_SIZE)

In [75]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [76]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [78]:
sampled_indices

array([58, 18, 24,  0, 47, 31, 12, 44, 50, 32, 49, 35, 31, 18,  6, 21,  0,
       32, 55, 29, 62, 11, 59, 50, 31, 17, 11, 18, 31,  5, 15, 63, 15, 42,
       51, 60,  2, 19, 59, 10, 28, 51,  9, 30, 38, 63, 57, 41, 18, 14, 23,
       16,  5, 17,  7, 60, 21, 53, 58, 41, 52, 53,  8, 41, 40, 23, 45, 31,
       64, 51, 41, 42, 27,  8, 48, 34, 33,  5, 51, 61,  2, 53,  1,  6, 17,
        7, 11,  7, 37, 55, 43, 49, 46, 33, 22, 14, 39, 30,  4, 41])

In [79]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "n whilst\n'Twixt you there's difference; but the fall of either\nMakes the survivor heir of all.\n\nAUFI"

Next Char Predictions: 
 "tFL\niS?flTkWSF,I\nTqQx;ulSE;FS'CyCdmv!Gu:Pm3RZyscFBKD'E-vIotcno.cbKgSzmcdO.jVU'mw!o ,E-;-YqekhUJBaR&c"


In [80]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174924


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_1'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=30

In [84]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 12s 68ms/step - loss: 2.6560
Epoch 2/30
172/172 [==============================] - 10s 61ms/step - loss: 1.9543
Epoch 3/30
172/172 [==============================] - 11s 61ms/step - loss: 1.6856
Epoch 4/30
172/172 [==============================] - 11s 62ms/step - loss: 1.5385
Epoch 5/30
172/172 [==============================] - 11s 63ms/step - loss: 1.4515
Epoch 6/30
172/172 [==============================] - 11s 63ms/step - loss: 1.3910
Epoch 7/30
172/172 [==============================] - 11s 63ms/step - loss: 1.3454
Epoch 8/30
172/172 [==============================] - 11s 64ms/step - loss: 1.3070
Epoch 9/30
172/172 [==============================] - 11s 65ms/step - loss: 1.2713
Epoch 10/30
172/172 [==============================] - 11s 65ms/step - loss: 1.2382
Epoch 11/30
172/172 [==============================] - 11s 65ms/step - loss: 1.2065
Epoch 12/30
172/172 [==============================] - 11s 66ms/step - loss: 1.1734
E

In [85]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_1/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [87]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [89]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I have been at Dicharding hell!
Dorset, what lives, Tyrand to the crown,
Who gave his body to my wife;
Lo, here thou didst prove by him when have I with thy
bring it to give him again to the body,
That shall be together weeping. I would he was bewrancius
Which hath his time of eyes. Churber so sweet a prince's day,
With love's power. Let such
For full of our own desire.

CORIOLANUS:
Turn these will efus business in the cestary.

BUCKINGHAM:
Now, if thou love me, that shall become to our laughs,
Are too late, or one dead little? What's the cunning turn,
For him tyrant banishment,
That they our leisure ise out.

MERCUTIO:
Out of heaven. Prepare to my teeter-hide inquirether.

YORK:
The cords of Tranio piercing blords.

ARCHIDAMUS:
Believe me, lords,
Be ruled her Sicily proge,
Too little gales of honour' both;
'Gawnets or old, Scroop,
Her father be either by great hands that place else hall, 'go.

TRANIO:
Such words I did rude as much beggure, so fornt I choose enmitable.

HENRY BO

In [90]:
print(generate_text(model, start_string=u"This is amazing "))

This is amazing array disless.

RIVERS:
But that's not queen? you are se on the world.

QUEEN MARGARET:
But thou art true, 'tis for the power of King of
The palme-place.

ORIOLANUS:
Well, sir, I do;
I do refer me to your successes.

Second Messenger:
A worthie thanks; but long have done
Great creatures tell me that you pay.

POLIXENES:
More noble loss!

GREMIO:
Trempeal to the fire, and prayers, alike:
This I can wish to-night,
Nor children, content to misis paragon ste with him.

Pedartimys,
Since at our hands;' it is your tribune.

SICINIUS:
Go all Pompey? well, I may make you:
Confire the clouds,
They love it is too good for nd.
I told you whose eldertainted to the worst is called for's!

PETRUCHIO:
The moonsworsel commonway thee! how love she is my love?
Prove the oratness reading thus sense too bitter
Upon the set of better perisper of my brain.

WARWICK:
True; Clarence! boy? cannot steal which often
With him to pry log as your children;
Worthily wrought by any secret midnight-goo